In [16]:
import pandas as pd

data_path = './datasets/sexism_data.csv'

# Carga los datos en un DataFrame
data = pd.read_csv(data_path)

# Muestra las primeras filas para entender la estructura
data.head()

,id,dataset,text,toxicity,sexist,of_id
0,0,other,MENTION3481 i didn't even know random was an o...,0.118180,False,-1
1,1,other,Bottom two should've gone! #mkr,0.251850,False,-1
2,2,callme,MENTION3111 MENTION3424 ladyboner deserves so ...,0.113331,False,-1
3,3,other,She shall now be known as Sourpuss #MKR #KatAn...,0.531153,False,-1
4,4,other,Tarah W threw a bunch of women under the bus s...,0.118718,False,-1


In [17]:
# Selecciona los datos del dataframe 'data' donde la columna 'sexist' es igual a True
processed_data = data[data['sexist'] == True]

# Elimina las columnas 'id', 'dataset', 'toxicity' y 'of_id' del dataframe 'processed_data'
processed_data = processed_data.drop(columns=['id', 'dataset', 'toxicity', 'of_id'])

# Reemplaza los caracteres no alfanuméricos en la columna 'text' del dataframe 'processed_data' por una cadena vacía
processed_data['text'] = processed_data['text'].str.replace(r"[^a-zA-Z0-9\s]", "", regex=True)

# Renombra la columna 'sexist' del dataframe 'processed_data' a 'class'
processed_data = processed_data.rename(columns={'sexist': 'class'})

# Asigna el valor 'sexism' a la columna 'class' del dataframe 'processed_data'
processed_data['class'] = 'sexism'

# Define la ruta de salida del archivo CSV
all_in_one_output_path = "dataset.csv"

# Guarda el dataframe 'processed_data' en un archivo CSV en la ruta especificada
processed_data.to_csv(all_in_one_output_path, index=False) 